<a href="https://colab.research.google.com/github/mephisto83/MagicWandPy/blob/main/MagicWand.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Start by connecting gdrive into the google colab

from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
source_folder = "/content/gdrive/Othercomputers/My Computer/output" #@param {type:"string"} "cfg/training/yolov7_pothole-tiny.yaml"
output_folder = "/content/gdrive/MyDrive/midas/backgroundRemoval" #@param {type:"string"} "cfg/training/yolov7_pothole-tiny.yaml"


In [ ]:
import os
from PIL import Image
import numpy as np

def save_numpy_as_png_with_alpha(image, file_name):
    # If the image has only 3 channels, add an alpha channel with full opacity
    if image.shape[2] == 3:
        alpha = np.ones((image.shape[0], image.shape[1], 1), dtype=np.uint8) * 255
        image = np.concatenate((image, alpha), axis=2)

    # Create a PIL image from the numpy array with mode 'RGBA'
    image_rgba = Image.fromarray(image, mode='RGBA')

    # Save the image as a PNG file
    image_rgba.save(file_name)

def compare_folder_files(folder_path1, folder_path2):
    # Get the list of files in each folder
    files1 = os.listdir(folder_path1)
    files2 = os.listdir(folder_path2)

    # Filter out any subdirectories in each folder
    files1 = [f for f in files1 if os.path.isfile(os.path.join(folder_path1, f))]
    files2 = [f for f in files2 if os.path.isfile(os.path.join(folder_path2, f))]

    # Find the files in folder1 that don't have matching names in folder2
    unique_files = [os.path.join(folder_path1, f) for f in files1 if f not in files2]

    # Return the list of unique file paths
    return unique_files

def get_file_name(file_path):
    # Get the base name of the file (i.e. the last part of the path)
    base_name = os.path.basename(file_path)

    # Return the base name, which includes the extension
    return base_name

In [ ]:
from PIL import Image, ImageChops

def magic_wand(file_path, tolerance=10, file_output_path=None):
    # Open the image file
    image = Image.open(file_path)

    # Find a color close to white in each of the four corners
    reference_colors = []
    for x in [0, image.width - 1]:
        for y in [0, image.height - 1]:
            quadrant = image.crop((x, y, x + image.width // 2, y + image.height // 2))
            reference_colors.append(quadrant.getpixel((0, 0)))

    # Create a blank image with the same size as the original image
    result = Image.new('RGBA', image.size, (0, 0, 0, 0))

    # Iterate over each reference color and mask
    for reference_color in reference_colors:
        # Create a blank mask for this reference color
        mask = Image.new('RGBA', image.size, (0, 0, 0, 0))

        # Use a queue to keep track of pixels to check
        queue = [(0, 0)]

        # Iterate over the queue until it's empty
        while queue:
            # Pop the next pixel from the queue
            x, y = queue.pop(0)

            # Skip if the pixel has already been processed
            if mask.getpixel((x, y))[3] != 0:
                continue

            # Get the color of the current pixel
            current_color = image.getpixel((x, y))

            # Calculate the difference in color between the current pixel and the reference color
            color_difference = sum([abs(reference_color[k] - current_color[k]) for k in range(3)])

            # If the difference in color is within the tolerance, set the pixel in the mask to white
            if color_difference <= tolerance:
                mask.putpixel((x, y), (255, 255, 255, 255))

                # Add neighboring pixels to the queue
                if x > 0:
                    queue.append((x - 1, y))
                if y > 0:
                    queue.append((x, y - 1))
                if x < image.size[0] - 1:
                    queue.append((x + 1, y))
                if y < image.size[1] - 1:
                    queue.append((x, y + 1))

        # Invert the mask to select the contiguous area
        mask = ImageChops.invert(mask)

        # Paste the original image onto the result image using the mask
        result.paste(image, (0, 0), mask)

    # Save the result image as a PNG if file_output_path is provided
    if file_output_path:
        result.save(file_output_path)

    return result


In [ ]:
files = compare_folder_files(source_folder, output_folder)
for i in range(len(files)): # 
  filename = files[i]
  output_file_path = os.path.join(output_folder, get_file_name(filename))
  tolerance = 40
  magic_wand(filename, tolerance, output_file_path)
